In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-11-23 14:51:41--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.201.29
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.201.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  46.4MB/s    in 35s     

2021-11-23 14:52:17 (44.7 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
!pip install gensim
!pip install scikit-learn

In [ ]:
import pickle
with open('/content/query_vocab.pkl', 'rb') as f:
    query_vocab = pickle.load(f)
with open('/content/collection_vocab.pkl', 'rb') as f:
    coll_vocab = pickle.load(f)
print(len(query_vocab), type(query_vocab))
print(len(coll_vocab), type(coll_vocab))

629 <class 'set'>
8422 <class 'set'>


In [ ]:
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
word_vectors = KeyedVectors.load_word2vec_format('/root/input/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
cosine_similarity([word_vectors['police']], [word_vectors['policeman']])

array([[0.6099762]], dtype=float32)

In [ ]:
for word in query_vocab:
    if word not in word_vectors:
        print(word)


grey
doughnut
selfie
<unk>


In [ ]:
i = 0
for word in coll_vocab:
    if word not in word_vectors:
        i += 1
        print(word)
print(i)

In [ ]:
import numpy as np
tprobs = {}
vocab = set([word for word in (query_vocab.union(coll_vocab)) if word in word_vectors])
i = 0
for word in query_vocab:
    print(i)
    i+=1
    if word in word_vectors:
        tprobs[word] = []
        for tword in vocab:
            tprobs[word].append((np.squeeze(cosine_similarity([word_vectors[word]], [word_vectors[tword]])).item(), tword))
        tprobs[word].sort(reverse=True)
        tprobs[word] = [np.array([x[0] for x in tprobs[word]]), [x[1] for x in tprobs[word]]]
        # tprobs[word][0] /= np.sum(tprobs[word][0])
        # tprobs[word] = {tprobs[word][1][i]:tprobs[word][0][i] for i in range(len(tprobs[word][1]))}

In [ ]:
P(q_t|trans) * P(trans|doc)

In [ ]:
len(tprobs)

624

In [ ]:
def get_top_k_words(tprobs, k):
    newprobs = {}
    for word in tprobs:
        newprobs[word] = [tprobs[word][0][:k], tprobs[word][1][:k]]
        newprobs[word][0] /= np.sum(newprobs[word][0])
        newprobs[word] = {newprobs[word][1][i]:newprobs[word][0][i] for i in range(len(newprobs[word][1]))}
    with open('/content/translation_probs_' + str(k) + '.pkl', 'wb') as f:
        pickle.dump(newprobs, f)

In [ ]:
get_top_k_words(tprobs, 5)
get_top_k_words(tprobs, 10)
get_top_k_words(tprobs, 15)
get_top_k_words(tprobs, 25)

In [ ]:
import pickle
import numpy as np

with open('/content/translation_probs_15.pkl', 'rb') as f:
    tprobs = pickle.load(f)

tprobs['dog']

{'beagle': 0.009021834942317895,
 'cat': 0.009253910626459387,
 'cats': 0.1531718094166442,
 'chihuahua': 0.03648044203050662,
 'collie': 0.15458145522471264,
 'dachshund': 0.03596379850452101,
 'dog': 0.012161064344699974,
 'dogs': 0.010556399958246893,
 'kitten': 0.15332606275590105,
 'pet': 0.03643400430853104,
 'pitbull': 0.03515267659486206,
 'puppies': 0.15280079727061746,
 'puppy': 0.009858281031757524,
 'rottweiler': 0.035341018974983505,
 'sheltie': 0.15589644401523872}

In [ ]:
word, tprobs[word]

('city', [array([ 1.00000012,  0.67237395,  0.61981499, ..., -0.1102897 ,
         -0.11678723, -0.13778095]),
  ['city',
   'town',
   'downtown',
   'district',
   'county',
   'neighborhood',
   'streets',
   'suburbs',
   'village',
   'area',
   'sewer',
   'waterfront',
   'sidewalks',
   'metro',
   'street',
   'streetlights',
   'park',
   'nightlife',
   'urban',
   'plaza',
   'state',
   'department',
   'lakefront',
   'community',
   'airport',
   'skyline',
   'region',
   'riverside',
   'zoo',
   'fairgrounds',
   'stadium',
   'cemetery',
   'transit',
   'harbor',
   'highway',
   'roads',
   'courthouse',
   'police',
   'expressway',
   'recreation',
   'local',
   'buildings',
   'hall',
   'cathedral',
   'port',
   'citizens',
   'marina',
   'south',
   'north',
   'skatepark',
   'sidewalk',
   'waterway',
   'west',
   'east',
   'mall',
   'university',
   'drainage',
   'bustling',
   'corridor',
   'taxis',
   'valley',
   'shire',
   'countryside',
   'pa

In [ ]:
with open('/content/translation_probs.pkl', 'wb') as f:
    pickle.dump(tprobs, f)